# Internet of Things

El microcontrolador que estamos usando es NodeMCU v1.0. ESP-12E.

Al encender el microcontrolador se carga el boot.py y luego el main.py.

Casi todos los main.py van a trabajar con un bucle while.

## NUNCA CONECTAR POSITIVO Y NEGATIVO CON UN CABLE!!!!!!!!

## Hello world:

In [ ]:
import machine
import utime

led = machine.Pin(2, machine.Pin.OUT)

while True:
    led.on()
    utime.sleep(1)
    led.off()
    utime.sleep(1)

## Estado del botón:

In [ ]:
import machine
import utime
led = machine.Pin(2, machine.Pin.OUT)
boton = machine.Pin(0, machine.Pin.IN)
while True:

    estado_boton = boton.value()
    print(estado_boton)
    led.value(estado_boton)
    utime.sleep(0.1)

## Pull.UP para definir voltaje a 0 y 1

In [ ]:
from machine import Pin
import utime

boton2 = Pin(16, Pin.PULL_UP)

while True:
    valor = boton2.value()
    print('valor: {}'.format(valor))
    utime.sleep(0.1)

# Hay que conectar un cable dupont al pin 16 y a tierra hace que el value sea 0

## Recreación salida analógica con una salida digital

Usamos PWM para dar una salida analógica mediante una digital. Nos permites más estados que 0 y 1.

In [ ]:
from machine import Pin, PWM
import utime

led = Pin(2, Pin.OUT)
led_pwm = PWM(led) # la frequencia se puede especificar ahí como segundo elemento de la tupla o por separado

led_pwm.freq(500)

while True:
    for i in range(1023):
        led_pwm.duty(i)
        utime.sleep_ms(3)
    for i in range(1023, 0, -1):
        led_pwm.duty(i)
        utime.sleep_ms(3)

Con PWM podemos usar servos y ccntrolar así el movimiento que hace, siendo `0` y `1` los valores límite y los intermedios, posiciones intermedias.

Podemos hacer lo mismo pero con un potenciómetro (que no tenemos) y depende del voltaje, da más intensidad o menos.

In [ ]:
from machine import Pin, PWM, ADC
import utime

led = Pin(2, Pin.OUT)
led_pwm = PWM(led)
led_pwm.freq(1000)

pot = ADC(0)

while True:
    lectura = pot.read()
    led_pwm.duty(lectura)
    print(lectura)
    utime.sleep(0.5)

## Los Timers:

Se usan con `machine.Timer()` y se usan para tareas recurrentes o para hacer contadores.

Con `Timer(-1)` tenemos un timer simulado por software y con otros números los de hardware.

Iniciamos con .init() y los parámetros son:
- `period`: El periodo, cada cuánto se activa
- `mode`: Tenemos el `ONE_SHOT`, que se activa una vez y el `PERIODIC`, que se activa cada los milisegundos indicados.
- `callback`: Podemos especificar un callback, usando funciones (incluyendo lambdas).

In [ ]:
from machine import Pin, PWM, ADC
import utime

tim = Timer(-1)

def texto(x): # x es el lugar en memoria de la instancia
    print('hola') # nunca poner un sleep aquí, ya que se romperá con el period de abajo

tim.init(period=1000, mode=Timer.PERIODIC, callback=texto)

Podemos usar `time.deinit()` para parar el timer. Si hacemos un sleep antes, el timer se va a estar ejecutando durante ese tiempo, lo que nos permite controlar las veces que se ejecuta.

In [ ]:
from machine import Pin, PWM, ADC, Timer
import utime

tim = Timer(-1)

def texto(x):
    print('hola')

tim.init(period=1000, mode=Timer.PERIODIC, callback=texto)

utime.sleep(3)

tim.deinit()

La técnica usada anteriormente para leer el botón se llama pulling. Lee el estado periódicamente del botón. No funciona bien con sleeps muy altos.

Podemos usar un timer para lanzar un callback cuando se pulse. Para ello se usa el método `.irq()`, que tiene dos parámetros:
- `función`: La función que debe ejecutar
- `trigger`: Define lo que hará que se lance la señal.
    - `IRQ_FALLING`: Pulsación del botón
    - `IRQ_RISING`: Soltar el botón

In [ ]:
from machine import Pin, PWM, ADC, Timer
import utime

def boton_callback(x):
    print('botón pulsado')

boton = Pin(0, Pin.IN)
boton.irq(boton_callback, trigger=Pin.IRQ_FALLING) # tráfico descendente

while True:
    estado = boton.value()
    print('x')
    utime.sleep(0.1)

Otra implementación:

In [ ]:
from machine import Pin, PWM, ADC, Timer
import utime

boton_pulsado= False

def boton_callback(x):
    global boton_pulsado
    boton_pulsado = True

boton = Pin(0, Pin.IN)
boton.irq(boton_callback, trigger=Pin.IRQ_FALLING) # tráfico descendente

while True:
    print('x')
    if boton_pulsado:
        print("boton_pulsado")
        boton_pulsado = False
    utime.sleep(0.1)

En el mundo real hay un montón de ruido al pulsar el botón y no es tan fácil capturar el raising y el falling. Se puede lidiar con ese problema de forma manual o usando librerías específicas. Usando el código anterior, nos metemos en el bucle:

In [ ]:
from machine import Pin, PWM, ADC, Timer
import utime

boton_pulsado = False
tiempo_inicio_boton = 0

def boton_callback(x):
    global boton_pulsado, tiempo_inicio_boton
    boton_pulsado = True
    tiempo_inicio_boton = utime.ticks_ms()    


boton = Pin(0, Pin.IN)
boton.irq(boton_callback, trigger=Pin.IRQ_FALLING)

while True:
    print("x")
    if boton_pulsado:
        print("boton pulsado marca {}".format(tiempo_inicio_boton))
        ahora =  utime.ticks_ms()  
        tiempo_boton = utime.ticks_diff(ahora, tiempo_inicio_boton)

        print(tiempo_boton)
        if tiempo_boton < 50:
            utime.sleep_ms(50-tiempo_boton)
        estado = boton.value()
        if estado:
            print("esto es ruido, no hacer nada")
        else:
            print("boton pulsado de verdad")
        utime.sleep(0.5) #antirrebotes
        boton_pulsado = False
    utime.sleep(0.1)

Para más implementaciones: http://docs.micropython.org/en/v1.9.3/pyboard/pyboard/tutorial/debounce.html

### Otras librerías interesantes:
- `machine`
- `utime`
- `esp`